In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os.path import join
folder = join('..', 'data', 'laws')

In [3]:
import fastText
import numpy as np
import re
from sklearn.neighbors import NearestNeighbors
import sklearn
import pymorphy2
import tqdm
import random
from sklearn.model_selection import train_test_split
morph = pymorphy2.MorphAnalyzer()

In [70]:
# очищает текст от знаков преминания
# возвращает массив из слов текста в нормальной форме

def process_text(text, dict_verbs={}):
    s = "".join(text)
    s = re.sub(r"[^A-Za-zА-Яа-я]+", " ", s)
    words = [word.lower() for word in s.split(" ")]   
    words_ = []
    for word in tqdm.tqdm_notebook(words):
        words_ += [morph.parse(word)[0].normal_form]
        part_of_sp = str(morph.parse(word)[0].tag).split(",")[0]
        if part_of_sp in ['VERB', 'PRTS', 'PRTF']:
            if words_[-1] in dict_verbs.keys():
                dict_verbs[words_[-1]].add(word)
            else:
                dict_verbs[words_[-1]] = set([word])
    return words_

In [4]:
texts = []
for i in range(1,26):
    texts += open(join(folder, '{}.txt'.format(i)), encoding='utf-8')

In [19]:
words = process_text(texts)

In [20]:
s = " ".join(words)
tmp = open('tmp.txt', mode='w')
tmp.write(s)

9225499

In [152]:
model = fastText.train_unsupervised('tmp.txt', dim=128)

In [153]:
words_ = open('tmp.txt').readline()
words_ = words_.split(" ")
words_ = np.unique(words_)
words_ = words_[1:]

In [154]:
X = [model.get_word_vector(word) for word in words_]

In [155]:
word2X = dict(zip(words_, X))

In [156]:
kNN = NearestNeighbors(n_neighbors=20).fit(X)

In [157]:
# нахождение 'синонимов' той же части речи

def find_syn(word):
    w = morph.parse(word)[0].normal_form
    try:
        ind = kNN.kneighbors([word2X[w]])[1]
    except KeyError:
        print("Данного слова нет в словаре")
        return
    for w_ in words_[ind].ravel():
        if morph.parse(w)[0].tag.cyr_repr.split(',')[0] == morph.parse(w_)[0].tag.cyr_repr.split(',')[0]:
            print(w_)

In [158]:
find_syn('видеонаблюдение')

видеонаблюдение
наблюдение
перекредитование
несоблюдение
самосовершенствование
видеоматериал
неподтверждение
видеоноситель
усовершенствование
тифлосурдопереводчик
здравоохранение
соблюдение
злоупотребление


**Классификация глаголов**

In [159]:
verbs = []

for word in words_:
    if morph.parse(word)[0].tag.cyr_repr.split(',')[0] == "ИНФ":
        verbs.append(word)
        print(word)

абортировать
адаптировать
адресовать
аккредитовать
аккумулировать
амортизировать
амортизироваться
анализировать
аннулировать
аннулироваться
арендовать
аттестовать
аффилировать
бальзамировать
бидти
блокировать
браковать
бросить
буксировать
быть
варить
ввезти
ввести
вводить
вводиться
ввозить
ведать
вернуть
вернуться
вести
вестись
взаимодействовать
взвесить
взимать
взиматься
взыскать
взыскивать
взыскиваться
взять
видеть
вкладывать
включать
включаться
включить
владеть
влечь
влиять
вложить
вменить
вменяться
вмешиваться
вмещать
внедрять
внести
вносить
вноситься
вовлекать
возбудить
возбуждать
возвести
возводить
возвратить
возвращать
возвращаться
возглавлять
возглавляться
возлагать
возлагаться
возложить
возместить
возмещать
возмещаться
возникать
возникнуть
возобновить
возобновлять
возобновляться
возражать
войти
воспитывать
воспользоваться
воспрепятствовать
воспрещаться
воспринимать
восстанавливать
восстанавливаться
восстановить
востребовать
вручать
вручаться
вручить
вскрыть
встать
встречать
вс

подтверждаться
подчиняться
позволять
позднооглохнуть
показать
показывать
покинуть
покрывать
покрыть
покупаться
полагать
полагаться
положить
получать
получаться
получить
пользоваться
поместить
помещать
помещаться
понести
понижать
понизить
понимать
пониматься
попадать
пополняться
порождать
поручать
поручаться
поручить
поселкообразовать
поселкообразуть
посещать
последовать
послужить
поставить
поставлять
поставляться
постановлять
построить
поступать
поступить
посчитать
потерять
потребить
потреблять
потребляться
потребовать
правоудостоверять
правоустанавливать
пребывать
превосходить
превысить
превышать
предлагать
предлагаться
предложить
предназначаться
предназначить
предоставить
предоставлять
предоставляться
предотвращать
предписать
предполагать
предполагаться
предпринять
представить
представлять
представляться
предупредить
предупреждать
предупреждаться
предусматривать
предусматриваться
предусмотреть
предшествовать
предъявить
предъявлять
предъявляться
прекратить
прекращать
прекращаться
прео

In [160]:
len(verbs)

1260

In [161]:
dict_words = dict(zip(verbs, np.zeros(len(verbs), dtype=int)))

In [162]:
with open('verbs.txt', 'r', encoding="utf-8") as f:
    for verb in f.readlines():
        print(verb)
        dict_words[verb[:-1]] = 1

способствовать

передать

обжаловать

выражать

аннулировать

закрепить

заключить

подготовить

прекращать

разъяснять

предоставить

истребовать

предъявлять

выполнять

уполномочить

применять

присвоить

созывать

заявлять

защитить

вносить

ограничивать

входить

признавать

руководить

инициировать

исполнять

выразить

передавать

налагаться

совершать

владеть

выдавать

приглашать

голосовать

обязать

уведомить

возобновить

налагать

анализировать

подготавливать

требовать

посещать

применить

предъявить

управлять

заявить

представить

вводиться

выплатить

выступать

заключать

оказывать

разработать

отклонить

пригласить

произвести

получить

готовить

удостоверять

составлять

дать

создавать

оцениваться

приостановить

защищать

избирать

заслушивать

перечислять

получать

обеспечивать

одобрить

возмещать

оформлять

заполнить

вводить

вступить

представлять

размещать

проводить

знакомить

оформить

установить

отменить

нести

продлить

проводиться

регистр

In [163]:
X = [model.get_word_vector(word) for word in verbs]

In [164]:
X_train, X_test, y_train, y_test = train_test_split(X, list(dict_words.values()), test_size=0.3)

Logistic Regression

In [165]:
from sklearn.linear_model import LogisticRegression


In [217]:
lr = LogisticRegression(C=100000, penalty='l1')
lr.fit(X_train, y_train)

LogisticRegression(C=100000, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [206]:
from sklearn.model_selection import GridSearchCV
parameters = {'penalty':('l1', 'l2'), 'C':[0.00001, 0.0001, 0.001, 0.01, 1, 10, 100, 1000, 10000, 100000]}
lr = LogisticRegression()
clf = GridSearchCV(lr, parameters, cv=5)
clf.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'penalty': ('l1', 'l2'), 'C': [1e-05, 0.0001, 0.001, 0.01, 1, 10, 100, 1000, 10000, 100000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [207]:
clf.best_params_, clf.best_score_

({'C': 1e-05, 'penalty': 'l1'}, 0.828798185941043)

In [218]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([731, 151], dtype=int64))

In [219]:
pred = lr.predict(X_train)

In [220]:
print(accuracy_score(pred, y_train))

0.854875283446712


In [221]:
y_pred = lr.predict(X_test)

In [222]:
from sklearn.metrics import *

In [223]:
accuracy_score(y_test, y_pred)

0.8148148148148148

In [224]:
print(np.unique(y_pred, return_counts=True))
print(np.unique(y_test, return_counts=True))
for i in range(len(y_pred)):
    if y_pred[i] == 1 and y_pred[i] == y_test[i]:
        print(i)

(array([0, 1]), array([344,  34], dtype=int64))
(array([0, 1]), array([326,  52], dtype=int64))
18
46
54
56
141
250
269
312


In [225]:
test_text = open(join(folder, 'test.txt'), encoding='utf-8').readlines()

In [173]:
DICT_OF_VERBS = {}
words_ = process_text(test_text, dict_verbs=DICT_OF_VERBS)

In [174]:
DICT_OF_VERBS.pop('дать')
DICT_OF_VERBS.pop('иметь')

{'имеет', 'имеют', 'имеющего', 'имеющие', 'имеющий', 'имеющих'}

In [175]:
verbs = []

for word in words_:
    if morph.parse(word)[0].tag.cyr_repr.split(',')[0] == "ИНФ":
        verbs.append(word)        
        if verbs[-1] in DICT_OF_VERBS.keys():
            DICT_OF_VERBS[verbs[-1]].add(word)
        else:
            DICT_OF_VERBS[verbs[-1]] = set([word])
        print(word)

принять
одобрить
иметь
обязать
сохранять
относиться
являться
проживать
определять
обеспечивать
регулировать
возникать
составлять
являться
использоваться
обеспечивать
составлять
сохранить
изменить
создать
обладать
иметь
создать
обладать
иметь
взаимодействовать
связать
объединить
подвергнуться
характеризоваться
сформировать
направить
характеризоваться
обеспечивать
приводить
использоваться
быть
использовать
иметь
оказывать
загрязнять
превышать
оказывать
обеспечиваться
сохраняться
установить
обеспечиваться
установить
соблюдаться
установить
обеспечиваться
сохраняться
загрязнять
определяться
объесть
загрязнять
определяться
объесть
установить
привести
установить
обеспечиваться
загрязнять
загрязнять
планировать
происходить
утратить
направить
предъявлять
определять
повлечь
иметь
вызвать
подпадать
включить
разрушать
озоноразрушать
определяться
озоноразрушать
озоноразрушать
озоноразрушать
озоноразрушать
озоноразрушать
содержаться
озоноразрушать
рекуперировать
озоноразрушать
озоноразрушать
озонора

причинить
допускаться
загрязнить
расположить
допускаться
загрязнить
допускаться
осуществляться
согласовать
обязать
соблюдать
ионизировать
допускать
ионизировать
информировать
принимать
обеспечивать
нести
запрещаться
облучить
разрешаться
провести
обосновать
облучить
осуществляться
облучить
устанавливаться
исходить
принимать
возвратить
образоваться
обеспечить
обязать
выполнять
использовать
принимать
запрещаться
подвергаться
запрещаться
создать
осуществлять
запрещаться
изменить
являться
соблюдаться
осуществлять
обязать
обеспечивать
разрабатывать
осуществлять
подлежать
быть
регулироваться
запрещаться
прилегать
быть
создать
использовать
утратить
озоноразрушать
дать
регулироваться
утратить
охранять
устанавливаться
оказывать
создаваться
регулироваться
обеспечиваться
устанавливаться
озоноразрушать
подлежать
объесть
озоноразрушать
вводиться
осуществлять
озоноразрушать
озоноразрушать
подлежать
объесть
озоноразрушать
осуществлять
озоноразрушать
устанавливаться
обязать
соблюдать
обязать
принимать


накопить
включить
накопить
накопить
осуществлять
накопить
проводить
осуществлять
накопить
включать
накопить
накопить
выполнить
накопить
устанавливаться
осуществлять
требовать
применяться
возникать
применяться
принять
установить
предусмотреть
применяться
вступать
признать
утратить
утрачивать
утрачивать
привести


In [176]:
verbs = np.unique(verbs)

In [177]:
len(verbs)

264

In [178]:
test = [model.get_word_vector(word) for word in verbs]

In [179]:
pred = lr.predict(test)

In [180]:
np.unique(pred)

array([0, 1])

In [181]:
predicted_verbs = []

In [182]:
for i in range(len(pred)):
    if pred[i] == 1:
        predicted_verbs.append(verbs[i])
        print(verbs[i])

включить
вносить
возобновлять
выдавать
выдать
дать
загрязнить
защищать
звереть
изменить
иметь
информировать
минерализовать
накопить
нарушить
обезвреживать
обеспечивать
обеспечить
планировать
предназначить
представить
предъявлять
привести
применять
провести
проводить
произвести
размещать
создавать
создать
требовать


In [183]:
[DICT_OF_VERBS[x] for x in predicted_verbs]

[{'включен',
  'включенная',
  'включенного',
  'включенному',
  'включенные',
  'включенных',
  'включены',
  'включить'},
 {'вносить', 'вносят'},
 {'возобновляемых', 'возобновлять'},
 {'выдавать'},
 {'выданных', 'выдать'},
 {'дать'},
 {'загрязненных', 'загрязнить'},
 {'защищать'},
 {'зверей', 'звереть'},
 {'изменена', 'измененный', 'изменить'},
 {'иметь'},
 {'информировать'},
 {'минерализованной', 'минерализовать'},
 {'накопить', 'накопленного', 'накопленный'},
 {'нарушенного', 'нарушенных', 'нарушить'},
 {'обезвреживать', 'обезвреживающие'},
 {'обеспечивает',
  'обеспечивать',
  'обеспечивают',
  'обеспечивающего',
  'обеспечивающей',
  'обеспечивающие',
  'обеспечивающими',
  'обеспечивающих'},
 {'обеспечено', 'обеспечить'},
 {'планировать', 'планируемой', 'планируемую', 'планируемых', 'планирующим'},
 {'предназначена', 'предназначить'},
 {'представить', 'представленные'},
 {'предъявляемые', 'предъявлять'},
 {'привести'},
 {'применяемые', 'применяемым', 'применяемых', 'применять'},

SVM

In [184]:
from sklearn.svm import SVC

In [185]:
svc = SVC(C=0.0001, kernel='linear')
svc.fit(X_train, y_train)

SVC(C=0.0001, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [186]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([731, 151], dtype=int64))

In [187]:
y_pred = svc.predict(X_test)

In [188]:
accuracy_score(y_test, y_pred)

0.8624338624338624

In [189]:
print(np.unique(y_pred, return_counts=True))
print(np.unique(y_test, return_counts=True))
for i in range(len(y_pred)):
    if y_pred[i] == 1 and y_pred[i] == y_test[i]:
        print(i)

(array([0]), array([378], dtype=int64))
(array([0, 1]), array([326,  52], dtype=int64))


In [190]:
from sklearn.model_selection import GridSearchCV
parameters = {'kernel':('linear', 'rbf'), 'C':[0.00001, 0.0001, 0.001, 0.01, 1, 10, 100, 1000, 10000, 100000]}
svc = SVC(gamma=0.1)
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'kernel': ('linear', 'rbf'), 'C': [1e-05, 0.0001, 0.001, 0.01, 1, 10, 100, 1000, 10000, 100000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [191]:
clf.best_params_

{'C': 1e-05, 'kernel': 'linear'}

In [192]:
clf.best_score_

0.828798185941043

In [193]:
clf.predict(X_train)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Записать результаты в файлик

In [235]:
def print_result(dict_verbs, predicted_verbs, file='result.txt'):
    f = open(file, mode='w')
    for verbs in [dict_verbs[x] for x in predicted_verbs]:
        for verb in verbs:
            f.write(verb + '\n')

In [236]:
print_result(DICT_OF_VERBS, predicted_verbs)